In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/toddgavin/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Add a flag for sarcasm by comparing the post text to https://nlds.soe.ucsc.edu/sarcasm1. Compare the text of the post with the sarcasm corpus and add a flag for detected or not.

# Sarcasm Text

In [2]:
sarcasmTextList = []

for i in range(1, 1000):
    
    try: 
        # Open file in read mode
        with open('data-sarc-sample/sarc/sarcastic_' + str(i) + '.txt', 'r') as file:
            # Read the entire contents of the file into a string variable
            sarcasm_text = file.read()
    
        sarcasmTextList.append(sarcasm_text)
        
    finally:
        continue

In [3]:
# Create a pandas DataFrame with the list of strings
df_sarcasm = pd.DataFrame({'Text': sarcasmTextList})

In [4]:
df_sarcasm

,Sarcasm Text
0,"Actually, they didn't. The whole tragedy was c..."
1,At your service: Comparison I could've jus...
2,"So which is it: the action is moral, the actio..."
3,Interesting how the study was set in Pittsburg...
4,"Ah, I see. Your reasons are secret reasons. ..."
...,...
992,and
993,"Ha, that is just an idiotic perspective. We'd ..."
994,So you are saying that despite the majority of...
995,"depends on your definition of ""human being."""


# Non-Sarcasm Text

In [5]:
notSarcasmTextList = []

for i in range(1, 1000):
    
    try: 
        # Open file in read mode
        with open('data-sarc-sample/notsarc/not_sarcastic_' + str(i) + '.txt', 'r') as file:
            # Read the entire contents of the file into a string variable
            not_sarcasm_text = file.read()
    
        notSarcasmTextList.append(not_sarcasm_text)
        
    finally:
        continue

In [6]:
# Create a pandas DataFrame with the list of strings
df_not_sarcasm = pd.DataFrame({'Text': notSarcasmTextList})

In [7]:
df_not_sarcasm

,Not Sarcasm Text
0,"This is a pretty touchy issue, and I agree wit..."
1,See above
2,"In other words, you think a Supreme Court deci..."
3,"um, yeah, you could say that... (won't argue....."
4,"Well, Google is your friend here - try fossil ..."
...,...
991,"Thanks, my friend, as you can see, Jito comple..."
992,What do you mean by this? Could we not have th...
993,And the answer is: we don't know. Maybe it cam...
994,And what would make them separate species? How...


# Implementing Flag Detection

In [8]:
df_master_dataset = pd.read_csv("../Master_Dataset_Raw.csv")

In [ ]:
# Step 1: Combine df_sarcasm and df_not_sarcasm into a single dataframe
df_sarcasm['label'] = 'sarcasm'
df_not_sarcasm['label'] = 'not sarcasm'
df = pd.concat([df_sarcasm, df_not_sarcasm])

# Step 2: Preprocess the text data
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['text'])
y = df['label']

# Step 3: Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Train a logistic regression model on the training set
clf = LogisticRegression()
clf.fit(X_train, y_train)

# Step 5: Evaluate the performance of the model on the test set
y_pred = clf.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Precision:', precision_score(y_test, y_pred, pos_label='sarcasm'))
print('Recall:', recall_score(y_test, y_pred, pos_label='sarcasm'))
print('F1 score:', f1_score(y_test, y_pred, pos_label='sarcasm'))

# Step 6: Use the trained model to predict the labels of the Narrative column in df_master_dataset
X_new = vectorizer.transform(df_master_dataset['Narrative'])
y_new = clf.predict(X_new)
df_master_dataset['label'] = y_new